# MLSS 2019: Optimal Transport and Wasserstein Distances

The goal of this first practical session is to introduce computational optimal transport (OT) in Python. You will familiarize yourself with OT by:
1. using the Python library POT (Python Optimal Transport),
2. coding Sinkhorn's algorithm.

In the second practical session, we will use optimal transport as a nice geometrical tool in machine learning.

<b>We are going to use Google Collab to run this notebook. In order to install all the necessary files run the following cells:</b>

In [ ]:
import os
!pip install --upgrade git+https://github.com/mlss-skoltech/tutorials.git#subdirectory=optimal_transport_tutorial

In [ ]:
# Check your installation by importing POT
!pip install pot
import ot

Declare ```DATA_PATH``` as a path to the data from the tutorial package

In [ ]:
import pkg_resources

DATA_PATH = pkg_resources.resource_filename('optimaltransport', 'data/')

<b>If you are running this notebook locally, make sure to clone the tutorial repository:</b>

```!pip install --upgrade git+https://github.com/mlss-skoltech/tutorials.git#subdirectory=optimal_transport_tutorial```



And install the following package:

* Install with pip: ```bash pip install pot```
* Install with conda: ```bash conda install -c conda-forge pot ```

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## 1. Solving Exact OT: Linear Programming

### Reminders on Optimal Transport

Optimal Transport is a theory that allows us to compare two (weighted) points clouds $(X, a)$ and $(Y, b)$, where $X \in \mathbb{R}^{n \times d}$ and $Y \in \mathbb{R}^{m \times d}$ are the locations of the $n$ (resp. $m$) points in dimension $d$, and $a \in \mathbb{R}^n$, $b \in \mathbb{R}^m$ are the weights. We ask that the total weights sum to one, i.e. $\sum_{i=1}^n a_i = \sum_{j=1}^m b_j = 1$.

The basic idea of Optimal Transport is to "transport" the mass located at points $X$ to the mass located at points $Y$.

Let us denote by $\mathcal{U}(a,b) = \left\{ P \in \mathbb{R}^{n \times m} \,|\, P \geq 0, \sum_{j=1}^m P_{ij} = a_i, \sum_{i=1}^n P_{ij} = b_j\right\}$ the set of admissible transport plans.

If $P \in \mathcal{U}(a,b)$, the quantity $P_{ij} \geq 0$ should be regarded as the mass transported from point $X_i$ to point $Y_j$. For this reason, it is called a *transport plan*.

We will also consider a *cost matrix* $C \in \mathbb{R}^{n \times m}$. The quantity $C_{ij}$ should be regarded as the cost paid for transporting one unit of mass from $X_i$ to $Y_j$. This cost is usually computed using the positions $X_i$ and $Y_j$, for example $C_{ij} = \|X_i - Y_j\|$ or $C_{ij} = \|X_i - Y_j\|^2$.

Then transporting mass according to $P \in \mathcal{U}(a,b)$ has a total cost of $\sum_{ij} P_{ij} C_{ij}$.

In "Optimal Transport", there is the word _Optimal_. Indeed, we want to find a transport plan $P \in \mathcal{U}(a,b)$ that will minimize its total cost. In other words, we want to solve
$$
    \min_{P \in \mathcal{U}(a,b)} \sum_{ij} C_{ij }P_{ij}.
$$

This problem is a Linear Program: the objective function is linear in the variable $P$, and the constraints are linear in $P$. We can thus solve this problem using classical Linear Programming algorithms, such as the simplex algorithm.

If $P^*$ is a solution to the Optimal Transport problem, we will say that $P^*$ is an optimal transport plan between $(X, a)$ and $(Y, b)$, and that $\sum_{ij} P^*_{ij} C_{ij}$ is the optimal transport distance between $(X, a)$ and $(Y, b)$: it is the minimal amount of "energy" that is necessary to transport the initial mass located at points $X$ to the target mass located at points $Y$.

### Computing Optimal "Croissant" Transport using POT

We will solve the Bakery/Cafés problem of transporting croissants from a number of Bakeries to Cafés in Moscow.

We use fictional positions, production and sale numbers (that both sum to the same value).

We have acess to the position of Bakeries $X \in \mathbb{R}^{8 \times 2}$ and their respective production $a \in \mathbb{R}^8$ which describe the source point cloud. The Cafés where the croissants are sold are defined by their position $Y \in \mathbb{R}^{5 \times 2}$ and $b \in \mathbb{R}^{5}$.

In [ ]:
# Load the data
import pickle

with open(DATA_PATH + 'croissants.pickle', 'rb') as file:
    croissants = pickle.load(file)

X = croissants['bakery_pos']
a = croissants['bakery_prod']
Y = croissants['cafe_pos']
b = croissants['cafe_prod']

print('Bakery productions =', a)
print('Café sales =', b)
print('Total number of croissants =', a.sum())

In [ ]:
plt.figure(figsize=(8,8))
plt.scatter(X[:,0], X[:,1], s=10*a, c='r', edgecolors='k', label='Bakeries')
plt.scatter(Y[:,0], Y[:,1], s=10*b, c='b', edgecolors='k', label='Cafés')
plt.legend(fontsize=20)
plt.axis('off')
plt.title('Moscow Bakeries and Cafés', fontsize=25)
plt.show()

Let us now compute the cost matrix $C \in \mathbb{R}^{n \times m}$. Here, we will use two different costs: $\ell_1$ and $\ell_2$ costs.

In [ ]:
C_1 = np.zeros((8,5)) # TODO: contains the l1 distances
C_2 = np.zeros((8,5)) # TODO: contains the l2 distances

We can now compute the Optimal Transport plan to transport the croissants from the bakeries to the cafés, for the two different costs.

In [ ]:
optimal_plan_1 = ot.emd() # TODO: compute the exact OT plan using function ot.emd
print(optimal_plan_1)
optimal_cost_1 = # TODO: compute the OT cost for the l1 ground cost
print('1-Wasserstein distance =', optimal_cost_1)
print('')

optimal_plan_2 = ot.emd() # TODO: compute the exact OT plan using function ot.emd
print(optimal_plan_2)
optimal_cost_2 = # TODO: compute the OT cost for the l2 ground cost
print('2-Wasserstein distance =', np.sqrt(optimal_cost_2))

In [ ]:
fig = plt.figure(figsize=(17,8))

ax = fig.add_subplot(1, 2, 1)
ax.scatter(X[:,0], X[:,1], s=10*a, c='r', edgecolors='k', label='Bakeries')
ax.scatter(Y[:,0], Y[:,1], s=10*b, c='b', edgecolors='k', label='Cafés')
# TODO: plot a line between Bakery i and Café j whenever some croissants are transported between i and j
ax.axis('off')
ax.set_title('$\ell_1$ cost', fontsize=30)

ax = fig.add_subplot(1, 2, 2)
ax.scatter(X[:,0], X[:,1], s=10*a, c='r', edgecolors='k', label='Bakeries')
ax.scatter(Y[:,0], Y[:,1], s=10*b, c='b', edgecolors='k', label='Cafés')
# TODO: plot a line between Bakery i and Café j whenever some croissants are transported between i and j
ax.axis('off')
ax.set_title('$\ell_2$ cost', fontsize=30)

plt.legend(fontsize=20)
plt.show()

## 2. Sinkhorn Algorithm for Entropy Regularized Optimal Transport

### Reminders on Sinkhorn Algorithm

In real applications, and especially in Machine Learning, we often have to deal with huge numbers of points. In this case, the linear programming algorithms which have cubic complexity will take too much time to run.

That's why in practise, among other reasons, people minimize another criterion given by
$$
    \min_{P \in \mathcal{U}(a,b)} \langle C, P \rangle + \epsilon \sum_{ij} P_{ij} [ \log(P_{ij}) - 1].
$$
When $\epsilon$ is sufficiently small, we can consider that a solution to the above problem (often refered to as "Entropy-regularized Optimal Transport") is a good approximation of a real optimal transport plan.

In order to solve this problem, one can remark that the optimality conditions imply that a solution $P_\epsilon^*$ necessarily is of the form $P_\epsilon^* = \text{diag}(u) \, K \, \text{diag}(v)$, where $K = \exp(-C/\epsilon)$ and $u,v$ are two non-negative vectors.

$P_\epsilon^*$ should verify the constraints, i.e. $P_\epsilon^* \in \mathcal{U}(a,b)$, so that
$$
    P_\epsilon^* 1_m = a \text{  and  } (P_\epsilon^*)^T 1_n = b
$$
which can be rewritten as
$$
    u \odot (Kv) = a \text{  and  } v \odot (K^T u) = b
$$

Then Sinkhorn's algorithm alternate between the resolution of these two equations, and reads
$$
    u \leftarrow \frac{a}{Kv} \text{  and  } v \leftarrow \frac{b}{K^T u}
$$

In [ ]:
def sinkhorn(a, b, C, epsilon=0.1, max_iters=100):
    """Run Sinnkhorn's algorithm"""
    
    # TODO: Compute the kernel matrix K
    
    # TODO: Alternate projections
    
    return # TODO

In [ ]:
np.round(sinkhorn(a, b, C_2/C_2.max(), epsilon=0.01), 2)

In [ ]:
optimal_plan_2

We first show that this algorithm is consistent with classical optimal transport, using the "croissant" transport example.

In [ ]:
plan_diff = []
distance_diff = []
for epsilon in np.linspace(0.01, 1, 100):
    optimal_plan_sinkhorn = # TODO: compute OT plan using Sinkhorn, with regularization strength epsilon
    optimal_cost_sinkhorn = # TODO: compute OT distance using Sinkhorn
    plan_diff.append() # TODO: compute the Frobenius distance between the exact OT plan and the Sinkhorn OT plan
    distance_diff.append() # TODO: compute the error between exact OT and Sinkhorn values (in %)

In [ ]:
plt.figure(figsize=(16,5))
plt.loglog(np.linspace(0.01, 1, 100), plan_diff, lw=4)
plt.xlabel('Regularization Strength $\epsilon$', fontsize=25)
plt.ylabel('$||P^* - P_\epsilon^*||_F$', fontsize=25)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.grid(ls='--')
plt.show()

In [ ]:
plt.figure(figsize=(16,5))
plt.loglog(np.linspace(0.01, 1, 100), distance_diff, lw=4)
plt.xlabel('Regularization Strength $\epsilon$', fontsize=25)
plt.ylabel('Error in %', fontsize=25)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.grid(ls='--')
plt.show()

Let us now compare the running time for sinkhorn and classical optimal transport algorithm on more data.

In [ ]:
n = 1000
m = 1000
d = 2

X = np.random.randn(n,d)
Y = np.random.randn(m,d)

a = np.ones(n)
b = np.ones(m)

C = np.zeros((n,m))
# TODO: compute the cost matrix (using l2 ground distance)

Because of Google Colab set up the time measuring can be unreliable, in order to get more certain results try running the code locally

In [ ]:
%time ot.emd(a,b,C)

In [ ]:
%time sinkhorn(a,b,C)

We see that sinkhorn is faster. What is even more interesting is that sinkhorn can be parallelerized on GPUs, giving further acceleration. Of course, Sinkhorn algorithm is not computing the exact optimal transport plan any more.

## 3. Optimal Transport in Dimension 1

In dimension $d=1$, computing OT boils down to sorting the points. You will check this fact, and discuss the influence of the regularization strength $\epsilon$.

In [ ]:
n = 4
m = 4

X = np.random.uniform(size=n)
Y = np.random.uniform(size=m)

a = np.ones(n)
b = np.ones(m)

plt.figure(figsize=(17,4))
plt.scatter(X, np.zeros(n), s=200*a, c='r')
plt.scatter(Y, np.zeros(m), s=200*b, c='b')
for i in range(n):
    plt.gca().annotate(str(i+1), xy=(X[i],0.005), size=30, color='r', ha='center')
for j in range(m):
    plt.gca().annotate(str(j+1), xy=(Y[j],0.005), size=30, color='b', ha='center')
plt.axis('off')
plt.show()

In [ ]:
# TODO: Compute the OT plan using sorting, POT, and Sinkhorn. Discuss the results and the running times.